In [1]:
import torch
import torchvision.transforms as transforms              
from torchvision.datasets import ImageFolder
import torchvision.models as models
from torchvision.models import Inception_V3_Weights
from torch.utils.data import DataLoader
import numpy as np
from torch.utils.data import Subset
from src.temperature_scaling import ModelWithTemperature
from src.raps import split_data_set, raps_scores, raps_classification, eval_aps

# load pre-trained model InceptionV3 and set mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.inception_v3(weights=Inception_V3_Weights.IMAGENET1K_V1).to(device)

#  Reprocess: Center Crop and then resize to 299*299
data_transform = transforms.Compose([
    transforms.CenterCrop(299),
    transforms.Resize(299), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
])

sorted_val_path = "D:\\Download\\ImageNet-1K\\Validation_Set\\sorted_ImageNet_val"
dataset = ImageFolder(root=sorted_val_path, transform=data_transform)

# Temperature Scaling
model.eval() # only use output.logits of Inception's output
subset_size = len(dataset) // 10
indices = np.random.choice(len(dataset), subset_size, replace=False)
subset_dataset = Subset(dataset, indices)
train_loader = DataLoader(subset_dataset, batch_size=32, shuffle=False, num_workers=4)

model_with_temp = ModelWithTemperature(model, temperature = 1.0).to(device)
model_with_temp.set_temperature(train_loader)
model_with_temp.eval()

# The number of times the experiment is going to be repeated
num_runs = 10

# error rate
alpha = 0.1
# Hyperparameter 
lambda_ = 0.2
k_reg = 4

# construct and evaluate repeatedly
all_avg_set_sizes = []
all_avg_coverages = []
print("\n")
print("RAPS Classification, Start!\n")


for i in range(num_runs):
    print(f"Running experiment {i+1}/{num_runs}...")

    # split dataset
    calib_dataset, test_dataset = split_data_set(dataset, random_seed=i)

    # load data set respectively
    calib_loader = DataLoader(calib_dataset, batch_size=32, shuffle=False,num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

    # calculate q_hat
    calib_scores, _ = raps_scores(model, calib_loader, alpha, lambda_, k_reg, device)
    q_hat = np.quantile(calib_scores, 1 - 0.1)  # calculate 1-alpha quantile
    print(f"q_hat = {q_hat}")

    # construct APS
    aps, aps_labels, true_labels = raps_classification(model, test_loader, q_hat, lambda_, k_reg, device)

    # evaluate APS
    avg_set_size, avg_coverage = eval_aps(aps_labels, true_labels)
    print(f"Average Prediction Set Size After APS in runs {i+1}: {avg_set_size}")
    print(f"Average Coverage Rate in runs {i+1}: {100 * avg_coverage:.2f}%\n")

    # record current result
    all_avg_set_sizes.append(avg_set_size)
    all_avg_coverages.append(avg_coverage)

# calculate the final average result
final_avg_set_size = np.mean(all_avg_set_sizes)
final_avg_coverage = np.mean(all_avg_coverages)
final_set_size_std = np.std(all_avg_set_sizes, ddof=0)
final_coverage_std = np.std(all_avg_coverages, ddof=0)

print(f"Final Average Prediction Set Size: {final_avg_set_size:.2f} ± {final_set_size_std:.2f}")
print(f"Final Average Coverage: {final_avg_coverage:.4f} ± {final_coverage_std:.4f}")

Before temperature - NLL: 1.066, ECE: 0.022
Optimal temperature: 0.965
After temperature - NLL: 1.056, ECE: 0.026


RAPS Classification, Start!

Running experiment 1/10...
q_hat = 0.9492043495178223
Total set size: 78843
Total coverage sets: 22506
Total samples amount: 25000
Average Prediction Set Size After APS in runs 1: 3.15372
Average Coverage Rate in runs 1: 90.02%

Running experiment 2/10...
q_hat = 0.9484826087951661
Total set size: 78327
Total coverage sets: 22492
Total samples amount: 25000
Average Prediction Set Size After APS in runs 2: 3.13308
Average Coverage Rate in runs 2: 89.97%

Running experiment 3/10...
q_hat = 0.9492087423801423
Total set size: 78897
Total coverage sets: 22530
Total samples amount: 25000
Average Prediction Set Size After APS in runs 3: 3.15588
Average Coverage Rate in runs 3: 90.12%

Running experiment 4/10...
q_hat = 0.9451721966266633
Total set size: 77536
Total coverage sets: 22411
Total samples amount: 25000
Average Prediction Set Size After APS

## Result
From the above test, following results can be collected :
- Final Average Prediction Set Size: **3.14** / 1000
- Final Average Coverage: **89.95%** ($\alpha$=0.1)